In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd


## Libaries Download

In [ ]:
# import nltk
# nltk.download("all")

In [ ]:
# pip install stanfordnlp

In [ ]:
# import stanfordnlp
# stanfordnlp.download('ur')

In [ ]:
# nlp = stanfordnlp.Pipeline(processors='tokenize', lang='ur')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from keras import layers
import nltk
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np 
import os 
import io 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Flatten, Dense
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.layers import Bidirectional, GRU
import gensim
from gensim.models import word2vec
from keras import models

## Tokenization function

In [ ]:
# def tokenization(data):
#     def token(data):
#         doc = nlp(data)
#         for i, sentence in enumerate(doc.sentences):
#             a = [f"{token.text}" for token in sentence.tokens]
#         return a
#     tokenized_text = []
#     for document in data:
#         docum = token(document)
#         tokenized_text.append(docum)
#     return tokenized_text

In [ ]:
# x_train = tokenization(documents)


## Import Dataset


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Urdu Corpus/Training-Urdu-Corpus.csv")
documents = train.iloc[:, 2]
y_train = train.iloc[:, 3]

In [ ]:
test_file = pd.read_csv("/content/drive/MyDrive/Urdu Corpus/Testing-Urdu-Corpus.csv")
x_test = test_file.iloc[:, 2]
y_test = test_file.iloc[:,3]

## unique words lists

In [ ]:
unique_words = [] 
for each_document in x_train:
    for word in each_document:
        if len(word)>2 and word not in unique_words:
            unique_words.append(word) # Add each unique word of length>2 to the list

## Deep Learning

In [ ]:
voc_size=18000

embedding_dims=300
filters=16
kernel_size=3
hidden_dims=250

In [ ]:
tokenizer = Tokenizer(num_words = voc_size)  
print(tokenizer.fit_on_texts(documents))                      #tokens of text
sequences = tokenizer.texts_to_sequences(documents)
print(sequences)    #texy to seqences
word_index = tokenizer.word_index
print(word_index)
print('Found %s unique tokens.' % len(word_index))

x_train_text = pad_sequences(sequences,maxlen=500)
print('shape of data tensor:', x_train_text.shape)
print('shape of label tensor:', y_train.shape)

None
[[364, 97, 93, 124, 1274, 874, 610, 1851, 874, 97, 93, 7, 1195, 97, 93, 1196, 38, 271, 1274, 874, 610, 1851, 874, 267, 53, 85, 1979, 1197, 1330, 48, 1418, 403, 329, 446, 170, 1711, 3701, 5458, 1125, 498, 129, 1851, 8, 15, 403, 150, 209, 1980, 267, 53, 403, 1274, 874, 150, 372, 8, 3702, 8335, 1198, 267, 53, 48, 210, 3257, 1007, 591, 159, 708, 566, 1851, 324, 403, 1274, 874, 404, 803, 174, 403, 1274, 874, 404, 610, 403, 8336, 803, 875, 1419, 2851, 5459, 330, 324, 403, 1199, 174, 3258, 1067, 403, 2852, 5460, 641, 2853, 4479, 1199, 566, 324, 1331, 97, 93, 15, 2358, 174, 1851, 97, 93, 223, 1195, 122, 1274, 874, 64, 5461, 567, 610, 1851, 874, 1712, 1604, 804, 1712, 5462, 874, 3259, 5463, 874, 610, 5464, 874], [5465, 674, 83, 91, 8337, 1332, 1852, 568, 8338, 2854, 164, 7, 349, 62, 34, 945, 536, 129, 405, 568, 554, 1420, 1853, 8339, 674, 83, 297, 55, 5465, 4480, 1332, 1852, 568, 1605, 5466, 4, 611, 642, 228, 1421, 946, 406, 417, 947, 141, 129, 5467, 5468, 55, 57, 8340, 1275, 592, 2359, 94

In [ ]:
from gensim.models import KeyedVectors
from gensim.models import word2vec
embeddings_dictionary = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/URDU VEC/urduvec_140M_100K_300d.bin', binary=True, encoding='utf-8')

embedding_matrix = np.zeros((voc_size, 300))
for word, index in tokenizer.word_index.items():
   if word in embeddings_dictionary:
    embedding_vector = embeddings_dictionary[word]
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
maxlen=500
model= Sequential()
model.add(Embedding(voc_size,embedding_dims,input_length=maxlen))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
history= model.fit(x_train_text,y_train,epochs=10,batch_size=16)

Epoch 1/10
40/40 [==============================] - 6s 124ms/step - loss: 0.7610 - accuracy: 0.5298
Epoch 2/10
40/40 [==============================] - 5s 125ms/step - loss: 0.6742 - accuracy: 0.5784
Epoch 3/10
40/40 [==============================] - 5s 123ms/step - loss: 0.5519 - accuracy: 0.7038
Epoch 4/10
40/40 [==============================] - 5s 124ms/step - loss: 0.4595 - accuracy: 0.8056
Epoch 5/10
40/40 [==============================] - 5s 123ms/step - loss: 0.4140 - accuracy: 0.8762
Epoch 6/10
40/40 [==============================] - 5s 124ms/step - loss: 0.2977 - accuracy: 0.9154
Epoch 7/10
40/40 [==============================] - 5s 123ms/step - loss: 0.1102 - accuracy: 0.9671
Epoch 8/10
40/40 [==============================] - 5s 123ms/step - loss: 0.0364 - accuracy: 0.9906
Epoch 9/10
40/40 [==============================] - 5s 124ms/step - loss: 0.0107 - accuracy: 0.9953
Epoch 10/10
40/40 [==============================] - 5s 124ms/step - loss: 0.0029 - accuracy: 0.9984

In [ ]:
model.save_weights('pre_trained_glove_model_dense.h5')

In [ ]:
sequences = tokenizer.texts_to_sequences(x_test)
x_test_text = pad_sequences(sequences,maxlen=500)
print('shape of data tensor:', x_test_text.shape)
print('shape of label tensor:', y_test.shape)

shape of data tensor: (262, 500)
shape of label tensor: (262,)


In [ ]:
model.load_weights('pre_trained_glove_model_dense.h5')
model.evaluate(x_test_text,y_test)

9/9 [==============================] - 0s 23ms/step - loss: 1.4129 - accuracy: 0.5611


[1.412881851196289, 0.5610687136650085]

## CNN

In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(voc_size, 300,input_length=500))
model1.add(layers.Conv1D(128, 3, activation='relu'))
model1.add(layers.GlobalMaxPooling1D())
model1.add(layers.Dense(128, activation='relu'))
model1.add(layers.Dense(1, activation='sigmoid'))
model1.layers[0].set_weights([embedding_matrix])
model1.layers[0].trainable = False
model1.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
history= model1.fit(x_train_text,y_train,epochs=10,batch_size=16)
model1.summary()

Epoch 1/10
40/40 [==============================] - 4s 77ms/step - loss: 0.7423 - accuracy: 0.5125
Epoch 2/10
40/40 [==============================] - 3s 76ms/step - loss: 0.6500 - accuracy: 0.6050
Epoch 3/10
40/40 [==============================] - 3s 76ms/step - loss: 0.5697 - accuracy: 0.7069
Epoch 4/10
40/40 [==============================] - 3s 77ms/step - loss: 0.4564 - accuracy: 0.8088
Epoch 5/10
40/40 [==============================] - 3s 77ms/step - loss: 0.3576 - accuracy: 0.8699
Epoch 6/10
40/40 [==============================] - 3s 76ms/step - loss: 0.2446 - accuracy: 0.9295
Epoch 7/10
40/40 [==============================] - 3s 76ms/step - loss: 0.1887 - accuracy: 0.9420
Epoch 8/10
40/40 [==============================] - 3s 76ms/step - loss: 0.1063 - accuracy: 0.9718
Epoch 9/10
40/40 [==============================] - 3s 76ms/step - loss: 0.0980 - accuracy: 0.9796
Epoch 10/10
40/40 [==============================] - 3s 76ms/step - loss: 0.1001 - accuracy: 0.9592
Model: "s

In [ ]:
model1.save_weights('pre_trained_glove_model_cnn.h5')

In [ ]:
model1.save_weights('pre_trained_glove_model_cnn.h5')
model1.load_weights('pre_trained_glove_model_cnn.h5')
model.evaluate(x_test_text,y_test)

9/9 [==============================] - 0s 22ms/step - loss: 1.4129 - accuracy: 0.5611


[1.412881851196289, 0.5610687136650085]

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(voc_size, embedding_dims,input_length=maxlen))
model2.add(layers.Conv1D(64, 3, activation='relu'))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(100, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.layers[0].set_weights([embedding_matrix])
model2.layers[0].trainable = False

model2.summary()
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history1= model2.fit(x_train_text,y_train,epochs=10,batch_size=16)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 300)          5400000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 498, 64)           57664     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               6500      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 5,464,265
Trainable params: 64,265
Non-trainable params: 5,400,000
_________________________________________________________________
Epoch 1/10
40/40 [==============================] - 3s 57ms/step - loss: 0.7141 - accuracy: 0.4828
Epoch 

In [ ]:
model2.save_weights('pre_trained_glove_model_cnn2.h5')

In [ ]:
model2.load_weights('pre_trained_glove_model_cnn2.h5')

model2.evaluate(x_test_text,y_test)

9/9 [==============================] - 0s 33ms/step - loss: 0.6811 - accuracy: 0.6260


[0.6810597777366638, 0.6259542107582092]

## LSTM

In [ ]:
model3 = Sequential()
model3.add(Embedding(voc_size,embedding_dims,input_length=maxlen))
model3.add(Dropout(0.2))
model3.add(LSTM(300))
model3.add(Dropout(0.3))
model3.add(Dense(100,activation='relu'))
model3.add(Dropout(0.4))
model3.add(Dense(1,activation='sigmoid'))

model3.layers[0].set_weights([embedding_matrix])
model3.layers[0].trainable = False


model3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model3.summary())

histroy3=model3.fit(x_train_text,y_train,epochs=10,batch_size=16)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          5400000   
_________________________________________________________________
dropout (Dropout)            (None, 500, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

In [ ]:
model3.save_weights('pre_trained_glove_model_lstm.h5')

In [ ]:
model1.load_weights('pre_trained_glove_model_cnn.h5')
model2.evaluate(x_test_text,y_test)

9/9 [==============================] - 0s 30ms/step - loss: 0.6811 - accuracy: 0.6260


[0.6810597777366638, 0.6259542107582092]

In [ ]:
loss, accuracy = model1.evaluate(x_train_text, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model1.evaluate(x_test_text, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

## Embedding

In [ ]:
maxlen=500
model= Sequential()
model.add(Embedding(voc_size,embedding_dims,input_length=maxlen))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
# model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
history= model.fit(x_train_text,y_train,epochs=10,batch_size=16)

Epoch 1/10
40/40 [==============================] - 8s 174ms/step - loss: 0.9346 - accuracy: 0.5533
Epoch 2/10
40/40 [==============================] - 7s 172ms/step - loss: 0.6871 - accuracy: 0.5690
Epoch 3/10
40/40 [==============================] - 7s 173ms/step - loss: 0.6241 - accuracy: 0.6411
Epoch 4/10
40/40 [==============================] - 7s 174ms/step - loss: 0.3550 - accuracy: 0.8793
Epoch 5/10
40/40 [==============================] - 7s 174ms/step - loss: 0.0426 - accuracy: 0.9922
Epoch 6/10
40/40 [==============================] - 7s 174ms/step - loss: 0.0148 - accuracy: 0.9937
Epoch 7/10
40/40 [==============================] - 7s 174ms/step - loss: 3.0425e-04 - accuracy: 1.0000
Epoch 8/10
40/40 [==============================] - 7s 174ms/step - loss: 3.1367e-05 - accuracy: 1.0000
Epoch 9/10
40/40 [==============================] - 7s 174ms/step - loss: 3.5369e-06 - accuracy: 1.0000
Epoch 10/10
40/40 [==============================] - 7s 175ms/step - loss: 6.4318e-07 - 

In [ ]:
model.save_weights('pre_trained_glove_model_cnn_emb.h5')

In [ ]:

model.load_weights('pre_trained_glove_model_cnn_emb.h5')
model.evaluate(x_test_text,y_test)

9/9 [==============================] - 0s 21ms/step - loss: 0.8094 - accuracy: 0.5458


[0.8094494938850403, 0.5458015203475952]

In [ ]:
model3 = Sequential()
model3.add(Embedding(voc_size,embedding_dims,input_length=maxlen))
model3.add(Dropout(0.2))
model3.add(LSTM(300))
model3.add(Dropout(0.3))
model3.add(Dense(100,activation='relu'))
model3.add(Dropout(0.4))
model3.add(Dense(1,activation='sigmoid'))

# model3.layers[0].set_weights([embedding_matrix])
# model3.layers[0].trainable = False


model3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model3.summary())

histroy3=model3.fit(x_train_text,y_train,epochs=10,batch_size=16)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 300)          5400000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_4 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 100)               30100     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                

In [ ]:
model3.save_weights('pre_trained_glove_model_lstm_emb.h5')

In [ ]:

model3.load_weights('pre_trained_glove_model_lstm_emb.h5')
model3.evaluate(x_test_text,y_test)

9/9 [==============================] - 6s 606ms/step - loss: 1.3746 - accuracy: 0.5725


[1.3745533227920532, 0.572519063949585]

In [ ]:
from keras.layers import Embedding, SimpleRNN
RNN = models.Sequential()
RNN.add(Embedding(voc_size,embedding_dims,input_length=700))
RNN.add(SimpleRNN(32, activation = 'relu'))
RNN.add(Dense(1, activation = 'sigmoid'))
RNN.summary()
RNN.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
history = RNN.fit(x_train_text, y_train, epochs = 10, batch_size = 16)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 700, 300)          5400000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                10656     
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 1)                 33        
Total params: 5,410,689
Trainable params: 5,410,689
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
40/40 [==============================] - 70s 163ms/step - loss: 0.6947 - accuracy: 0.5445
Epoch 2/10
40/40 [==============================] - 7s 165ms/step - loss: 0.6413 - accuracy: 0.6819
Epoch 3/10
40/40 [==============================] - 6s 161ms/step - loss: 0.5346 - accuracy: 0.7541
Epoch 4/10
40/40 [==============================] - 7s 163ms/step - loss:

In [ ]:
RNN.save_weights('pre_trained_glove_model_RNN_emb.h5')
RNN.load_weights('pre_trained_glove_model_RNN_emb.h5')
RNN.evaluate(x_test_text,y_test)

9/9 [==============================] - 1s 45ms/step - loss: 1.0647 - accuracy: 0.6031


[1.0647403001785278, 0.6030534505844116]